<a href="https://colab.research.google.com/github/viktoriapatouna/Thesis-work-2025/blob/main/LIRmotifs_small_human_proteins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install biopython

In [ ]:
pip install tabulate

In [ ]:
import os
import csv
import regex
from Bio import SeqIO
from tabulate import tabulate
from collections import defaultdict

In [ ]:
 '''READ THE FASTA FILES AND RETURNS A DICTIONARY
    WHERE KEYS ARE PEPTIDES IDs AND VALUES ARE SEQUENCES'''
def read_fasta_file(file_path):
    sequences = {}

    with open(file_path, "r") as file:
        #Use Biopython's SeqIO to parse each FASTA record
        for record in SeqIO.parse(file, "fasta"):  #Choosing a specific FASTA record from the file
            sequences[record.id] = str(record.seq)  #Store ID as keys and sequence as value
    return sequences


'''READS A CSV FILE CONTAINING LIR MOTIF DATA AND RETURNS
  A DICTIONARY WITH UNIQUE MOTIF IDENTIFIERS AND MOTIF INFO AS KEY-VALUE PAIRS'''
def read_csv_file(file_path):
  sequences={}

  with open(file_path , "r" , newline="") as file:
    reader= csv.reader(file)
    headers = next(reader) #Skip the header row
    for row in reader:
      #Choose the Homo sapiens and canonical LIR motifs only
      if row[0].strip() == "Homo sapiens" and row[4].strip() != "Non canonical LIR" :
        uniprot_id = row[1].strip()
        start_position = row[8].strip()
        uniqueID =  f"{uniprot_id}_{start_position}" # Make a unique ID for each LIR motif combining Uniprot ID and start position


        #Extract and reconstruct LIR motif sequence
        if len(row) > 6 :
         motif= row[6].strip() # Remove any spaces and get the LIR motifs only
         if len(row)> 5 :
           motif_row5 = row[5].strip()
           motif_from_row5 = motif_row5[-2:] #Get last 2 amino acids from the sequence before the LIR motif position
           motif = motif_from_row5 + motif


        if row[11].strip()=="NO" or row[11].strip() =="N/A":
         experimentally_verified = "NO"
        else:
         experimentally_verified = "YES"


        #Store only if motif exists
        if motif:
           sequences[uniqueID] = {"motif" : motif , "verified" : experimentally_verified}
        else :
         print(f"Skipping empty motif in row :{row}")

  return sequences



  '''MATCHES HUMAN LIR MOTIFS FROM set1 AGAINST SMALL HUMAN PEPTIDES FROM set2
  AND RETURNS A DICTIONARY WHERE KEYS ARE MOTIF IDs AND VALUES ARE MATCHE DETAILS'''

def find_protein_matches(set1, set2):
    results = {}

    for uniqueID, motif1 in set1.items():
        motif_seq = motif1["motif"]
        experimentally_verified = motif1["verified"]
        motif_length = len(motif_seq)
        matches = []

        for id2, protein2 in set2.items():
          #Build Regex pattern with <=1 mismatch
          motif_pattern="({}){{e<=1}}".format(motif_seq)
          found_matches = []


          #Search for all matches within the peptide sequence
          for match in regex.finditer(motif_pattern, protein2  ,overlapped=True) :
            #search for all matches and returns match objects, .findall() only returns strings
            position= match.start()
            sequence = match.group()
            sequence_length = len(sequence)
            #Store match details(matched position in protein,matched sequence and its length)
            found_matches.append((position,sequence,sequence_length))

          #If any matches found for peptides,add them in found_matches variable
          if found_matches and 10 <= len(protein2) <= 100:
            matches.append({
                    "protein_id": id2,
                    "protein_length": len(protein2),  # length of the peptide from me set2
                    "matches" :found_matches,
                    "experimentally_verified" : experimentally_verified

                })


        #If this motif matched in any peptide, store the results under the motif's unique ID
        if matches:
            results[uniqueID] = matches

    return results

In [ ]:
#Read motif data from CSV file
set1 = read_csv_file("LIRcentral.csv")
#Read small human peptides sequence from FASTA file
set2 = read_fasta_file("Smallhumanpeptides.fasta")

#Check if either input set is empty and stop the execution
if not set1 or not set2:
   print("Error: One or both sets are empty.")

else:
    #Find where motifs matched peptides sequence
    matches = find_protein_matches(set1, set2)

    #Loop through all motifs that matched
    for uniqueID, match_list in matches.items():
        motif_length = len(set1[uniqueID]["motif"])  # motif length from the set1
        print(f"Protein: {uniqueID} (Length: {motif_length})")

        #Loop through all peptides sequences that had a match with this motif
        for match in match_list:
            print(f"  Found in: {match['protein_id']} (Length: {match['protein_length']})")
            #Loop through individual match positions and sequences
            for position, sequence ,*_ in match["matches"]:
               print(f"  Matches at position {position} : {sequence}")

Protein: A0A3Q9JIX6_160 (Length: 6)
  Found in: IP_059505|TX=9606 (Length: 54)
  Matches at position 18 : TSLSRL
  Found in: IP_060960|TX=9606 (Length: 33)
  Matches at position 1 : TSSRL
  Found in: IP_063073|TX=9606 (Length: 32)
  Matches at position 6 : TSYSRH
  Found in: IP_072017|TX=9606 (Length: 77)
  Matches at position 60 : TSSRL
  Found in: IP_073753|TX=9606 (Length: 40)
  Matches at position 19 : TSYSRG
  Found in: IP_073772|TX=9606 (Length: 37)
  Matches at position 4 : TYSRL
  Found in: IP_074407|TX=9606 (Length: 46)
  Matches at position 8 : TSYSLL
  Found in: IP_078763|TX=9606 (Length: 42)
  Matches at position 37 : TSSRL
  Found in: IP_083038|TX=9606 (Length: 42)
  Matches at position 15 : TSYSVL
  Found in: II_085042|TX=9606 (Length: 81)
  Matches at position 17 : TSYRL
  Found in: IP_085463|TX=9606 (Length: 68)
  Matches at position 10 : TSSRL
  Found in: IP_085490|TX=9606 (Length: 55)
  Matches at position 9 : TSYSIL
  Found in: IP_086869|TX=9606 (Length: 52)
  Matche

In [ ]:

'''LOAD INPUT DATASET'''

#Read motif data from CSV file
set1 = read_csv_file("LIRcentral.csv")
#Read small human peptides sequence from FASTA file
set2 = read_fasta_file("Smallhumanpeptides.fasta")

'''VALIDATE INPUT DATA'''

#Check if either input set is empty and stop the execution
if not set1 or not set2:
    print("Error: One or both sets are empty.")
else:
  #Find where motifs matched peptides sequence
  matches= find_protein_matches(set1,set2)

  #Table creation to store results from output
  table_info =[]
  #Set to store peptide's IDs that contain identical matches to motifs
  identical_with_LIRmotifs = set()

  #Loop through each motif from the CSV
  for uniqueID, match_list in matches.items():
      motif_length = len(set1[uniqueID]["motif"]) #Length of the current motif

      #Loop through each peptide that matched this motif
      for match in match_list:
        protein_id= match["protein_id"]
        protein_length = match["protein_length"]
        experimentally_verified = match["experimentally_verified"]


        #Loop through each individual match found in this peptide
        for position, sequence,sequence_length in match["matches"]:

          #Check if the matched sequence is identical to the motif
          if sequence == set1[uniqueID]["motif"] :
            identical= "YES"
            identical_with_LIRmotifs.add(protein_id) #Store the identical matches
          else:
            identical = "NO"

          #Append all relevant data to the result table
          table_info.append([uniqueID, motif_length,experimentally_verified,protein_id, protein_length, position,sequence,sequence_length, identical])

  headers = ["Motif ID", "Motif Length" , "Experimentally Verified" , "Matched in Protein" , "Protein Length" , "Matched Position" , "Matched Sequence","Matched Sequence Length", "Identical Sequence"]
  #print(tabulate(table_info, headers=headers, tablefmt= "grid"))

  #Write the results to a txt file in tab-separated format
  with open("output.txt", "w") as out:
    #out.write(tabulate(table_info, headers=headers, tablefmt= "grid"))
    for match_lir in table_info:
      #out.write(str(match_lir)+"\n")
      out.write('\t'.join(map(str,match_lir)) + '\n')

In [ ]:
'''RESYLTS ANALYSIS'''


# File paths for input and output files
csv_file = "LIRcentral.csv"
output_file = "output.txt"
information_file = "analysis_results.txt"


#Initialize sets to store unique human LIRmotifs data

Homo_sapiens_unique_name_motifs = set() #Unique motif entries with Uniprot ID and position
Homo_sapiens_motifs = set() #Unique Uniprot IDs with LIR motifs

# Dictionaries to store analysis results
lir_motif_counts = defaultdict(int)  # Total number of matches per LIR motif
lir_motif_proteins = defaultdict(set)  # Peptides matched per LIR motif
lir_motif_all_proteins = set() # All peptides with at least one motif match
lir_motif_deletions = defaultdict(int)  # Counts deletions, motif matches <6 amino acids long
lir_motif_insertions = defaultdict(int)  # Counts insertions, motif matches >6 amino acids long
lir_motif_substitutions = defaultdict(int)  # Counts substitutions , motif matches exactly 6 amino acids long but not identical
lir_motif_identical = defaultdict(int) # Matched sequences identical with the LIRmotifs

# Counter objects
total_matches = 0
deletions = 0
insertions = 0
substitutions = 0
identicalseq = 0


'''READ MOTIF ENTRIES FROM THE ORIGINAL CSV FILE
AND COLLECT HUMAN LIR MOTIF IDENTIFIERS'''

with open(csv_file , "r" , newline="") as file :
  reader = csv.reader(file)
  header= next(reader) # Skip CSV header

  for row in reader:
        if row[0].strip() == "Homo sapiens":
            uniprot_id = row[1].strip()
            start_position = row[8].strip()
            uniqueID = f"{uniprot_id}_{start_position}"
            Homo_sapiens_unique_name_motifs.add(uniqueID)
            Homo_sapiens_motifs.add(uniprot_id)


# Read the matches from output.txt file
with open(output_file, "r") as file:
    for line in file:
        cols = line.strip().split("\t")
        if len(cols) < 9:
            continue  # Skip incomplete rows

        lir_motif = cols[0]  # LIR motif ID
        protein_id = cols[3]  # Protein ID where the motif matched
        match_length = int(cols[7])  # Length of the matched sequence to classify mismatch type

        # Count matches with <= 1 mismatch (each row represents one)
        total_matches += 1

        # Count different motifs and their associated peptides
        lir_motif_counts[lir_motif] += 1
        lir_motif_proteins[lir_motif].add(protein_id)
        lir_motif_all_proteins.add(protein_id)

        # Classify type of match
        if cols[8]=="NO": #Not an identical match
          if match_length < 6:
            lir_motif_deletions[lir_motif] += 1
            deletions += 1
          elif match_length > 6:
            lir_motif_insertions[lir_motif] += 1
            insertions += 1
          else:  # match_length == 6
            lir_motif_substitutions[lir_motif] += 1
            substitutions += 1
        else : #Identical match
          lir_motif_identical[lir_motif] += 1
          identicalseq += 1

# Find the LIR motif with the highest number of matches
most_common_motif = max(lir_motif_counts, key=lir_motif_counts.get, default=None)
most_common_motif_count = lir_motif_counts.get(most_common_motif, 0)
#Count how many different peptides matched this motif
unique_proteins_for_motif = len(lir_motif_proteins.get(most_common_motif, set()))

# Find LIR motifs that appear at least 1 time
frequent_motifs = {
    motif: {
        "count": lir_motif_counts[motif],
        "unique_proteins": len(lir_motif_proteins[motif]),
        "deletions": lir_motif_deletions[motif],
        "insertions": lir_motif_insertions[motif],
        "substitutions": lir_motif_substitutions[motif],
        "identical" : lir_motif_identical[motif]
    }
    for motif in lir_motif_counts if lir_motif_counts[motif] >= 1
}

# Write all summary results to a TXT file
with open(information_file, "w") as file:
    file.write(f"1. Homo sapiens LIR motifs: {len(Homo_sapiens_unique_name_motifs)}\n")
    file.write(f"2. Different human proteins with LIR motifs: {len(Homo_sapiens_motifs)}\n")
    file.write(f"3. Total LIRmotifs-small human proteins mismatches: {total_matches}\n")
    file.write(f"4. Identified different LIR motifs: {len(lir_motif_counts)}\n")
    file.write(f"6. Mismatches in different proteins:{len(lir_motif_all_proteins)}\n")
    file.write(f"5. Deletions (<6): {deletions}\n")
    file.write(f"7. Insertions (>6): {insertions}\n")
    file.write(f"8. Substitutions (=6): {substitutions}\n")
    file.write(f"9. Identical : {identicalseq}\n")
    file.write(f"10. Most frequent LIR motif: {most_common_motif} ({most_common_motif_count} times)\n")
    file.write(f"11. Number of different proteins matched: {unique_proteins_for_motif}\n\n")
    file.write("LIR motifs matching x times:\n")
    for motif, details in frequent_motifs.items():
        file.write(f"   - {motif}: {details['count']} times\n")
        file.write(f"     Different proteins matched: {details['unique_proteins']}\n")
        file.write(f"     Deletions: {details['deletions']}, Insertions: {details['insertions']}, Substitutions: {details['substitutions']}, Identical:{details['identical']}\n\n")

In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from collections import defaultdict

# === Step 1: Parse output.txt for all match details ===
protein_matches = defaultdict(list)  # protein_id -> list of (motif_id, position, match_type)

with open("output.txt", "r") as file:
    next(file)  # Skip header
    for line in file:
        cols = line.strip().split("\t")
        if len(cols) < 9:
            continue
        motif_id = cols[0].strip()
        protein_id = cols[3].strip()
        position = cols[5].strip()
        identical = cols[8].strip()

        # Classify match type
        match_len = int(cols[7])
        if identical == "YES":
            match_type = "Identical"
        elif match_len < 6:
            match_type = "Deletion"
        elif match_len > 6:
            match_type = "Insertion"
        else:
            match_type = "Substitution"

        protein_matches[protein_id].append((motif_id, position, match_type))

# === Step 2: Load full sequences from FASTA ===
all_proteins = {}
with open("Smallhumanpeptides.fasta", "r") as fasta_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        all_proteins[record.id] = str(record.seq)

# === Step 3: Build FASTA records with descriptive headers ===
fasta_records = []

for protein_id, match_list in protein_matches.items():
    if protein_id not in all_proteins:
        print(f"Warning: {protein_id} not found in FASTA.")
        continue

    seq = all_proteins[protein_id]
    # Join all match annotations into one description line
    details = [f"{motif}@{pos}({typ})" for motif, pos, typ in match_list]
    description = "Matches: " + "; ".join(details)

    record = SeqRecord(
        Seq(seq),
        id=protein_id,
        description=description
    )
    fasta_records.append(record)

# === Step 4: Write to FASTA ===
with open("matched_proteins_with_details.fasta", "w") as out_fasta:
    SeqIO.write(fasta_records, out_fasta, "fasta")

print(f"Saved {len(fasta_records)} matched protein records with match types and positions to 'matched_proteins_with_details.fasta'")


Saved 2138 matched protein records with match types and positions to 'matched_proteins_with_details.fasta'
